In [1]:
import scipy.signal
import scipy
import numpy as np
import tifffile as tf
import matplotlib.pyplot as plt
%matplotlib qt


In [42]:
# end goal is to render a xy, yz, xz video of a worm neural time series. yz and xz must be interpolated
fname = 'C:/Users/rldun/Desktop/20200204-10-46-53_shortened.tiff'
#fname = 'C:/Users/rldun/Desktop/20200204-10-46-53.tiff'
fname = 'C:/Users/rldun/Box/Lab 2.0/data/20200225_RLD_1/20200225-11-41-45.tiff'
d = tf.imread(fname)
zsize = 10
hyperstack = np.zeros((d.shape[0]//zsize, zsize, d.shape[1], d.shape[2]), dtype=np.uint16)
for t in range(d.shape[0]):
    hyperstack[t//zsize, t%zsize, :,:] = d[t,:,:]
    
hyperstack.shape

(2000, 10, 220, 668)

In [ ]:
# get mips along xy, xz, and yz
xy = np.max(hyperstack[0,:,:,:], axis=0)
yz = np.max(hyperstack[0,:,:,:], axis=2)
xz = np.max(hyperstack[0,:,:,:], axis=1)
plt.imshow(xz)

In [ ]:
# example of 1-d interpolation applied on a per-pixel basis
from scipy.interpolate import interp1d

# example interpolation
x = np.linspace(0, 10, num=11, endpoint=True)
y = np.cos(-x**2/9.0)
f = interp1d(x, y)
f2 = interp1d(x, y, kind='cubic')
xnew = np.linspace(0, 10, num=41, endpoint=True)
# plot
#import matplotlib.pyplot as plt
#plt.plot(x, y, 'o', xnew, f(xnew), '-', xnew, f2(xnew), '--')
#plt.legend(['data', 'linear', 'cubic'], loc='best')
#plt.show()

# our interpolation
x = 290
z = 2
#plt.plot(xz[:, x])
vec = xz[:,x]
multiplier = 3.5
xvals = np.linspace(0, np.floor(len(vec) * multiplier), num=np.floor(len(vec)*multiplier))

# get interpolation function
xvals = np.linspace(0, zsize, num=zsize)
yvals = vec
f = interp1d(xvals, yvals, kind='cubic')
xvals2 = np.linspace(0, len(vec), num=int(multiplier*len(vec)))
yvals2 = f(xvals2)
#plt.plot(xvals, yvals)
#plt.plot(xvals2, yvals2)

# apply interpolation on every pixel
multiplier = 5
xsize = xz.shape[1]
zsize = 10
newzlength = int(multiplier * zsize)
output_arr = np.zeros((newzlength, xsize))
for x in range(xsize):
    yvals = xz[:,x]
    xvals = np.linspace(0, zsize, num=zsize)
    f = interp1d(xvals, yvals, kind='cubic')
    newx = np.linspace(0, zsize, num=newzlength)
    newy = f(newx)
    output_arr[:,x] = newy
    
plt.imshow(output_arr)

In [ ]:
from scipy.interpolate import interp2d

# apply 2d image interpolation to each frame to get xz/yz
def make_interp_frame(img, new_xsize=None, new_ysize=None):
    
    # input checking
    if new_xsize==None:
        new_xsize = img.shape[1]
    if new_ysize==None:
        new_ysize = img.shape[0]
    
    # set local vars
    curr_xsize = img.shape[1]
    curr_ysize = img.shape[0]
    z = img
    
    # if we're not actually doing any interpolation
    if new_xsize == curr_xsize and new_ysize == curr_ysize:
        return img
    elif new_xsize < curr_xsize or new_ysize < curr_ysize:
        raise(Exception('Error in make_interp_frame: Downsampling not tested!'))
    
    # build linear pixel coords
    x = np.linspace(0, curr_xsize, num=curr_xsize)
    y = np.linspace(0, curr_ysize, num=curr_ysize)
    
    # build interpolation function
    f = interp2d(x, y, z, kind='cubic')
    
    # apply to new linear pixel coordinates
    xnew = np.linspace(0, curr_xsize, num=new_xsize)
    ynew = np.linspace(0, curr_ysize, num=new_ysize)
    return f(xnew, ynew)
    

multiplier = 6
res = make_interp_frame(xz, new_ysize=int(zsize*multiplier))
#plt.imshow(res)
#plt.imshow(xz)

In [43]:
# make xzt and yzt images
multiplier = 4
xyt = np.max(hyperstack, axis=1)
yzt = np.max(hyperstack, axis=3)
xzt = np.max(hyperstack, axis=2)

# fill out new interpolated data
new_zsize = zsize * multiplier
new_xzt = np.zeros((xzt.shape[0], new_zsize, xzt.shape[2]))
new_yzt = np.zeros((yzt.shape[0], new_zsize, yzt.shape[2]))

for t in range(xyt.shape[0]):
    new_frame = make_interp_frame(xzt[t,:,:], new_ysize=new_zsize)
    new_xzt[t,:,:] = new_frame
    
    new_frame = make_interp_frame(yzt[t,:,:], new_ysize=new_zsize)
    new_yzt[t,:,:] = new_frame
    

In [44]:
# assemble image including xyt, xzt, and yzt
xsize = xyt.shape[2]
ysize = xyt.shape[1]
output_arr = np.zeros((xyt.shape[0], ysize + new_zsize, xsize + new_zsize), dtype=np.float32)
output_arr[:,:ysize, :xsize] = xyt
output_arr[:,ysize:, :xsize] = new_xzt
output_arr[:,:ysize, xsize:] = np.swapaxes(new_yzt, 1, 2)
print('new_xzt.shape: {}'.format(new_xzt.shape))
print('new_yzt.shape: {}'.format(new_yzt.shape))
print('xyt.shape: {}'.format(xyt.shape))
print('output_arr.shape: {}'.format(output_arr.shape))

new_xzt.shape: (2000, 40, 668)
new_yzt.shape: (2000, 40, 220)
xyt.shape: (2000, 220, 668)
output_arr.shape: (2000, 260, 708)


In [45]:
# save video
import imageio
# recast array for mip
#tosave = frames.reshape((vols_to_grab, zsize, ysize, xsize)).max(axis=1) 
# grab a frame somewhat in
tosave = output_arr
f0 = tosave.flatten()
f0.sort()
# take avg of top % pixelsf0
mymax = np.median(f0[-int(xsize * ysize * 0.01):])
mymin = np.amin(f0)
# compress to MIP, scale to min/max
tosave[tosave > mymax] = mymax
#tosave[tosave < mymin] = mymin
tosave = ((tosave - mymin) / (mymax - mymin) * 255)
# save Zx speedup (cause that's a nice round number?)
multiplier = 2
imageio.mimwrite(fname + '_{}x.mp4'.format(multiplier * zsize), tosave.astype(np.uint8), fps=multiplier * 1000//30, quality=6)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (708, 260) to (720, 272) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
